In [1]:
def cal_result(exp_name, criterion='best'):
    import dill as pickle
    data_stream = 70
    week_n = 7000 

    with open('data/y.pkl', 'rb') as f:
        y = pickle.load(f).flatten().tolist()

    for stream in range(1, data_stream + 1): 
        pre = (stream - 1) * 7000
        
        flag = False 
        start, end = pre + week_n, pre + 2 * week_n
        for j in range(start, end): 
            if y[j] == 1: 
                flag = True; break 

        def read_score(exp_name, week): 
            lst = []
            exp_name = os.path.join(os.path.join("exp", exp_name), exp_name)
            with open(exp_name + "Week" + str(week) +  "_result.txt", "r") as f: 
                sc = float(f.readline()) 
                while sc != None: 
                    lst.append(sc)

                    s = f.readline()
                    # while !s: s = f.readline()
                    if s == '': break  
                    sc = float(s) 

            return lst                

        if flag : 
            x_score = read_score(exp_name, stream)
            from sklearn.metrics import roc_curve, auc
            GT = y[start: end]
            from sklearn.metrics import roc_auc_score
            AUC = roc_auc_score(GT, x_score)
            print(stream, " week :", AUC)  

            # print(len(x_score), GT) 
            FPR, TPR, thresholds = roc_curve(GT, x_score)
            maxindex = (TPR-FPR).tolist().index(max(TPR-FPR))
            threshold = thresholds[maxindex]
            tpr, fpr = TPR[maxindex], FPR[maxindex]
            print("tpr: %f , fpr: %f, thresh: %f\n" % (tpr, fpr, threshold))
            # if criterion == 'bset': 
                # JD = [x - y for x, y in zip(TPR, FPR)]
            


In [2]:
cal_result("exp_icarl_all_data")

22  week : 0.9987139182623607
tpr: 1.000000 , fpr: 0.002572, thresh: 1.979269

23  week : 0.8198569384835479
tpr: 0.900000 , fpr: 0.277253, thresh: 0.793132

24  week : 0.8668957289429731
tpr: 1.000000 , fpr: 0.342305, thresh: 0.727732

25  week : 0.804691977077364
tpr: 0.800000 , fpr: 0.283381, thresh: 0.745373

26  week : 0.7447168458781361
tpr: 0.960000 , fpr: 0.560860, thresh: 0.613218

27  week : 0.7997575670448012
tpr: 0.727273 , fpr: 0.275158, thresh: 0.708054

28  week : 0.8170724070738437
tpr: 0.615385 , fpr: 0.114208, thresh: 0.834942

29  week : 0.7845873898165237
tpr: 0.641026 , fpr: 0.193794, thresh: 0.730195

30  week : 0.8364267236069478
tpr: 0.809524 , fpr: 0.236275, thresh: 0.713139

31  week : 0.8498026556537195
tpr: 0.772727 , fpr: 0.187033, thresh: 0.712227

32  week : 0.7567527498804401
tpr: 0.766667 , fpr: 0.327403, thresh: 0.635046

33  week : 0.8125019473923637
tpr: 0.826087 , fpr: 0.285653, thresh: 0.629386

34  week : 0.8393118279569892
tpr: 0.800000 , fpr: 0.